<a href="https://colab.research.google.com/github/Devanshu0502/NFL-Big-Data-Bowl-2023/blob/main/Introduction_Pandera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandera==0.13.4

In [2]:
import pandas as pd
from tabulate import tabulate
week_df = pd.read_csv('/content/drive/MyDrive/nfl-big-data-bowl-2023/week8.csv')
week_df.head()

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event
0,2021102800,189,37077.0,1,2021-10-29T00:27:23.000,18.0,ARI,right,21.38,6.94,0.0,0.0,0.00,43.74,223.19,None
1,2021102800,189,37077.0,2,2021-10-29T00:27:23.100,18.0,ARI,right,21.38,6.94,0.0,0.0,0.00,44.67,243.81,None
2,2021102800,189,37077.0,3,2021-10-29T00:27:23.200,18.0,ARI,right,21.38,6.95,0.0,0.0,0.00,45.69,303.24,None
3,2021102800,189,37077.0,4,2021-10-29T00:27:23.300,18.0,ARI,right,21.38,6.94,0.0,0.0,0.00,46.44,285.89,None
4,2021102800,189,37077.0,5,2021-10-29T00:27:23.400,18.0,ARI,right,21.38,6.95,0.0,0.0,0.01,47.99,341.60,None


In [3]:
import pandera as pa
from pandera.typing import DateTime, Float, Int, Series, String


class WeekSchema(pa.SchemaModel):
    # We coerce the field so it tries to convert if necessary
    game_id: Series[Int] = pa.Field(coerce=True, nullable=True, alias="gameId")
    play_id: Series[Int] = pa.Field(coerce=True, nullable=True, alias="playId")
    nfl_id: Series[Float] = pa.Field(coerce=True, nullable=True, alias="nflId")

    # Let's try to coerce frame_id, which is an integer, to float so we can the result later
    frame_id: Series[Float] = pa.Field(coerce=True, nullable=True, alias="frameId")
    time: Series[DateTime] = pa.Field(coerce=True, nullable=True, alias="time")
    jersey_number: Series[Float] = pa.Field(coerce=True, nullable=True, alias="jerseyNumber")
    team: Series[String] = pa.Field(coerce=True, nullable=True, alias="team")

    # With Check.isin we want to make sure it only contains a string "left" or "right"
    playDirection: Series[String] = pa.Field(coerce=True, nullable=True, alias="playDirection", isin=["right", "left"])

In [4]:
# In the results you'll see that frameId has become a Float
validated_schema = WeekSchema.validate(week_df)
validated_schema.head()

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event
0,2021102800,189,37077.0,1.0,2021-10-29 00:27:23.000,18.0,ARI,right,21.38,6.94,0.0,0.0,0.00,43.74,223.19,None
1,2021102800,189,37077.0,2.0,2021-10-29 00:27:23.100,18.0,ARI,right,21.38,6.94,0.0,0.0,0.00,44.67,243.81,None
2,2021102800,189,37077.0,3.0,2021-10-29 00:27:23.200,18.0,ARI,right,21.38,6.95,0.0,0.0,0.00,45.69,303.24,None
3,2021102800,189,37077.0,4.0,2021-10-29 00:27:23.300,18.0,ARI,right,21.38,6.94,0.0,0.0,0.00,46.44,285.89,None
4,2021102800,189,37077.0,5.0,2021-10-29 00:27:23.400,18.0,ARI,right,21.38,6.95,0.0,0.0,0.01,47.99,341.60,None


In [5]:
data = []
for col in validated_schema.columns:
    data.append([
        col,
        type(week_df[col][0]),
        type(validated_schema[col][0])
    ])
print(tabulate(data, headers=["Column","Datatype before validation","Datatype after validation"]))

Column         Datatype before validation    Datatype after validation
-------------  ----------------------------  --------------------------------------------------
gameId         <class 'numpy.int64'>         <class 'numpy.int64'>
playId         <class 'numpy.int64'>         <class 'numpy.int64'>
nflId          <class 'numpy.float64'>       <class 'numpy.float64'>
frameId        <class 'numpy.int64'>         <class 'numpy.float64'>
time           <class 'str'>                 <class 'pandas._libs.tslibs.timestamps.Timestamp'>
jerseyNumber   <class 'numpy.float64'>       <class 'numpy.float64'>
team           <class 'str'>                 <class 'str'>
playDirection  <class 'str'>                 <class 'str'>
x              <class 'numpy.float64'>       <class 'numpy.float64'>
y              <class 'numpy.float64'>       <class 'numpy.float64'>
s              <class 'numpy.float64'>       <class 'numpy.float64'>
a              <class 'numpy.float64'>       <class 'numpy.float64'>
di

In [6]:
from pandera.errors import SchemaError

# Just see what goes wrong
class WeekSchemaWithError(WeekSchema):
    # playDirection only contains left or right, so what if we only allow values which aren't there
    playDirection: Series[String] = pa.Field(coerce=True, nullable=True, alias="playDirection", isin=["up", "down"])


try:
    wrong_validated_schema = WeekSchemaWithError.validate(week_df)
except SchemaError as exc:
    print('Something went totally wrong')
    print(exc)

Something went totally wrong
<Schema Column(name=playDirection, type=DataType(str))> failed element-wise validator 0:
<Check isin: isin({'down', 'up'})>
failure cases:
         index failure_case
0            0        right
1            1        right
2            2        right
3            3        right
4            4        right
...        ...          ...
978944  978944        right
978945  978945        right
978946  978946        right
978947  978947        right
978948  978948        right

[978949 rows x 2 columns]


In [7]:
games_df = pd.read_csv('/content/drive/MyDrive/nfl-big-data-bowl-2023/players.csv')
games_df.head()

,nflId,height,weight,birthDate,collegeName,officialPosition,displayName
0,25511,6-4,225,1977-08-03,Michigan,QB,Tom Brady
1,28963,6-5,240,1982-03-02,"Miami, O.",QB,Ben Roethlisberger
2,29550,6-4,328,1982-01-22,Arkansas,T,Jason Peters
3,29851,6-2,225,1983-12-02,California,QB,Aaron Rodgers
4,30078,6-2,228,1982-11-24,Harvard,QB,Ryan Fitzpatrick


In [8]:
try:
    schema = pa.DataFrameSchema({
        "weight": pa.Column(
            int, pa.Hypothesis.two_sample_ttest(
                "QB", "T",
                groupby="officialPosition",
                relationship="greater_than",
                alpha=0.05
            )
        ),
        "officialPosition": pa.Column(str)
    })

    schema.validate(games_df)
except SchemaError as exc:
    print(f"Hypothesis test failed {exc}")

Hypothesis test failed <Schema Column(name=weight, type=DataType(int64))> failed series or dataframe validator 0:
<Check two_sample_ttest: failed two sample ttest between 'QB' and 'T'>


In [9]:
schema = pa.DataFrameSchema({
    "weight": pa.Column(
        int, pa.Hypothesis.two_sample_ttest(
            "QB", "T",
            groupby="officialPosition",
            relationship="less_than",
            alpha=0.05
        )
    ),
    "officialPosition": pa.Column(str)
})

schema.validate(games_df)

,nflId,height,weight,birthDate,collegeName,officialPosition,displayName
0,25511,6-4,225,1977-08-03,Michigan,QB,Tom Brady
1,28963,6-5,240,1982-03-02,"Miami, O.",QB,Ben Roethlisberger
2,29550,6-4,328,1982-01-22,Arkansas,T,Jason Peters
3,29851,6-2,225,1983-12-02,California,QB,Aaron Rodgers
4,30078,6-2,228,1982-11-24,Harvard,QB,Ryan Fitzpatrick
...,...,...,...,...,...,...,...
1674,53991,6-1,320,NaN,NaN,DT,Forrest Merrill
1675,53994,6-5,300,NaN,NaN,C,Ryan McCollum
1676,53999,6-4,312,NaN,NaN,DT,Jack Heflin
1677,54006,6-6,330,NaN,NaN,T,Jake Curhan
